<img src="./img/header.png">
# Globus SDK
https://github.com/globus/globus-sdk-python


# Globus SDK Docs
http://globus.github.io/globus-sdk-python/


# Requirements
- You need to be in the tutorial users group for sharing: https://www.globus.org/app/groups/50b6a29c-63ac-11e4-8062-22000ab68755
- Installed Globus Python SDK

In [ ]:
from __future__ import print_function # for python 2
tutorial_endpoint_1 = "ddb59aef-6d04-11e5-ba46-22000b92c6ec"      # endpoint "Globus Tutorial Endpoint 1"
tutorial_endpoint_2 = "ddb59af0-6d04-11e5-ba46-22000b92c6ec"      # endpoint "Globus Tutorial Endpoint 2"
tutorial_users_group = "50b6a29c-63ac-11e4-8062-22000ab68755"     # group "Tutorial Users"

# Configuration

First you will need to configure the client with an OAuth2 access token. For the purpose of this tutorial, you can obtain access tokens via the tokens.globus.org website. Click the "Jupyter Notebook" option and copy the resulting text below, or click on "Globus CLI" and copy the resulting text into ~/.globus.cfg.

In [ ]:
transfer_token = None # if None, tries to get token from ~/.globus.cfg file

# Transfer Client

The Globus Python SDK makes transfer functionality avaialable via a TransferClient class. 

## Instantiate Transfer Client

The TransferClient can be instantiated by passing in a token or by using the token configured in your home directory (~/.globus.cfg). Tokens can be obtained via the tokens.globus.org site. 

In [ ]:
import globus_sdk

tc = globus_sdk.TransferClient(token=transfer_token)

## Help

Get help on any method.

In [ ]:
help(tc.get_endpoint)

## Using the client

The transfer client makes REST resources avaialble via easy to use methods. 

The response from these methods wraps the HTTP response status, content type, text and JSON response body. 

In [ ]:
r = tc.get_endpoint(tutorial_endpoint_1)
print("HTTP Status Code:", r.http_status)
print("Content Type:", r.content_type)
print("Data:", repr(r.data))
print("Endpoint Display Name:", r["display_name"]) # shortcut for r.data["display_name"]

Helper methods for APIs that returns lists have iterable responses, and automatically take care of paging where required:

In [ ]:
r = tc.endpoint_search(filter_scope="my-endpoints")
for ep in r:
    print("{} ({})".format(ep["display_name"], ep["id"]))

## Handling errors

If the API returns an error response (HTTP status code 4xx or 5xx), it will be translated to a Python exception and raised:

In [ ]:
try:
    r = tc.get_endpoint("dcb2e10e-de27-4b99-8722-1a69aa3fc467")
except globus_sdk.exc.GlobusAPIError as ex:
    print("HTTP Status Code:", ex.http_status)
    print("Error Code      :", ex.code)
    print("Error Message   :", ex.message)

There are five basic classes of errors:

1. Bad request - there is something wrong with the request from the client, like a mispelled parameter name or missing required data. These errors have a code that starts with ``BadRequest`` or ``ClientError.BadRequest``.
2. State conflict - this is a very broad category, and covers all the errors that can happen during normal operation, and neither the client nor the server could have anticipated and avoided the error. Examples: local filesystem permissions not allowing the requested path on a remote GridFTP endpoint, endpoint not found (could have been deleted concurrently by another client). This also includes network errors communicating with GridFTP endpoints and other external services. These errors typically have a code containing ``PermissionDenied``, ``Conflict``, or ``ExternalError``.
3. Network error - network failure between the REST client and the REST API server. These errors will result in a ``globus_sdk.NetworkError`` being raised by the SDK.
4. Planned downtime - code ``ServiceUnavailable``.
5. Server error - caused by a bug in the REST API server (code ``ServerError.InternalError``). We log such errors and incorperate fixes into our next release, but developers are still encouraged to submit details to the mailing list when they encounter these errors. Note that sometimes these errors are actually a sign of a bad request type error, i.e. the bug in the server is that it's not anticipating the exact type of bad data, so it's not reporting the correct error code, but the problem can still be resolved by a change to the client.

# Endpoint management

## Endpoint search

Globus has over 8000 registered endpoints. To find endpoints of interest you can access powerful search capabilities via the SDK. For example, to search for a given string across the descriptive fields of endpoints (names, description, keywords):

In [ ]:
search_str = "Globus Tutorial Endpoint"
endpoints = tc.endpoint_search(search_str)
print("==== Displaying endpoint matches for search: '{}' ===".format(search_str))
for ep in endpoints:
    print("{} ({})".format(ep["display_name"], ep["id"]))

## Restricting search scope with filters

There are also a number of default filters to restrict the search for 'my-endpoints', 'my-gcp-endpoints',     'recently-used', 'in-use', 'shared-by-me','shared-with-me') 

In [ ]:
endpoints = tc.endpoint_search(
        filter_fulltext=search_str, filter_scope="recently-used")
for ep in endpoints:
    print("{} ({})".format(ep["display_name"], ep["id"]))

## Endpoint details

You can also retrieve complete information about an endpoint, including name, owner, location, and server configurations. 

In [ ]:
endpoint = tc.get_endpoint(tutorial_endpoint_1)
print("Display name:", endpoint["display_name"])
print("Owner:", endpoint["owner_string"])
print("ID:", endpoint["id"])

# File operations

## Path encoding & UTF gotchas

## Autoactivate Endpoints
Globus endpoints must be "activated" before they can be used, which means associating an a credential with the endpoint that is used for login to that endpoint. Before performing operations against an endpoint, you should "autoactivate" the endpoint. On Globus Connect Personal and Shared endpoints, autoactivation will automatically create the necessary credentials to access the endpoint. For endpoints that require activation (e.g., those with a MyProxy or MyProxy OAuth identity provider) you can activate those endpoints via the Globus website. Here we autoactivate the Globus tutorial endpoints using their endpoint_ids.

In [ ]:
#help(tc.endpoint_activate)
ep1result = tc.endpoint_autoactivate(tutorial_endpoint_1)
ep2result = tc.endpoint_autoactivate(tutorial_endpoint_2)
print("Response code:", ep1result["code"])
print("Response message:", ep1result["message"])

## Get a directory listing

Having activated an endpoint, you can now perform operations on it. For example, performing an ls command to retrieve directory contents. 

In [ ]:
# help(tc.operation_ls)
endpoint_id = tutorial_endpoint_1
endpoint_path = "/share/godata/"
r = tc.operation_ls(endpoint_id, path=endpoint_path)
print("==== Endpoint_ls for endpoint {} {} ====".format(endpoint_id, endpoint_path))
for item in r:
    print("{}: {} [{}]".format(item["type"], item["name"], item["size"]))

## Make directory

You can create a new directory.

In [ ]:
#help(tc.operation_mkdir)
try:
    endpoint_id = tutorial_endpoint_1
    endpoint_path = "/~/tutorial_dir"
    mkdir_result = tc.operation_mkdir(endpoint_id, path=endpoint_path)
    print(mkdir_result["message"])
except globus_sdk.exc.GlobusAPIError as ex:
    print("Error:", ex.message)

## Rename

You can rename files and directories on your endpoints. 

In [ ]:
#help(tc.operation_rename)
try:
    endpoint_id = tutorial_endpoint_1
    r = tc.operation_rename(endpoint_id, oldpath="/~/tutorial_dir",
                            newpath="/~/tutorial_dir_renamed")
    print(r["message"])
except globus_sdk.exc.GlobusAPIError as ex:
    print("Error:", ex.message)

# Task submission and management

The Globus task interface allows you to create and manage asynchronous file transfer and deletion tasks. 

## Transfer

Creating a transfer is a two stage process. First you must create a description of the data you want to transfer (which also creates a unique submission_id), and then you can submit the request to Globus to transfer that data. 

If the submit_transfer fails, you can safely resubmit the same transfer_data again. The submission_id will ensure that this transfer request will be submitted once and only once.

In [ ]:
# help(tc.submit_transfer)
source_endpoint_id = tutorial_endpoint_1
source_path = '/share/godata/'

dest_endpoint_id = tutorial_endpoint_2
dest_path = '/~/'

label = "My tutorial transfer"

transfer_items = []

## Recursively transfer source path contents
transfer_items.append(
    tc.make_submit_transfer_item(source_path,
                                 dest_path,
                                 recursive=True))
## Alternatively, transfer a specific file
# transfer_items.append(
#     tc.make_submit_transfer_item("/source/path/file.txt",
#                                  "/dest/path/file.txt"))

# Ensure endpoints are activated
tc.endpoint_autoactivate(tutorial_endpoint_1)
tc.endpoint_autoactivate(tutorial_endpoint_2)

# make_submit_transfer_data() automatically gets a submission_id for once-and-only-once submission
transfer_data = tc.make_submit_transfer_data(
    source_endpoint_id,
    dest_endpoint_id,
    transfer_items,
    label=label)
transfer_result = tc.submit_transfer(transfer_data)
task_id = transfer_result["task_id"]
print("Task ID:", task_id)

## Get Task By ID

While the task is running, or after completion, you can get information that describes the transfer task. 

In [ ]:
r = tc.get_task(transfer_result.data['task_id'])
print ("Label:", r["label"])
print ("Status:", r["status"])
print ("Transfer: {} -> {}".format(r["source_endpoint_display_name"],
                                   r["destination_endpoint_display_name"]))
    
if r.data["status"] == "SUCCEEDED":
    print ("Bytes transferred:", r["bytes_transferred"])
    print ("Files transferred:", r["files_transferred"])
    print ("Transfer rate:", r["effective_bytes_per_second"], "bps")

## Check destination endpoint

After the transfer has finished you can list the contents of the destination endpoint

In [ ]:
ls_iter = tc.operation_ls(dest_endpoint_id, path=dest_path)
print("==== Endpoint_ls for endpoint {} {} ====".format(dest_endpoint_id, dest_path))
for item in ls_iter:
    print("{}: {} [{}]".format(item["type"], item["name"], item["size"]))

## Get task list

You can get a list of past or current tasks with the following call. Note, you can also use a variety of filters to control the scope of results. 

In [ ]:
# help(tc.task_list)
r = tc.task_list(num_results=10)
for i, item in enumerate(r):
    print(item.data['status'],
          item.data['task_id'], 
          item.data['type'],
          item.data['source_endpoint_display_name'],
          item.data['destination_endpoint_display_name'],
          item.data['label'])

## Filter task list

Retrieve only active tasks.  TODO: Link to filters. 

In [ ]:
r = tc.task_list(num_results=10, filter="status:ACTIVE")
for i, item in enumerate(r):
    print(item.data['status'],
          item.data['task_id'], 
          item.data['type'],
          item.data['source_endpoint_display_name'],
          item.data['destination_endpoint_display_name'],
          item.data['label'])

## Cancel task

You can also cancel a running task. 

In [ ]:
#help(tc.cancel_task)
r = tc.cancel_task(task_id)
print ("{}: {}".format(r["code"], r["message"]))

## Get event list for task

Every task stores periodic event markers (e.g., errors, performance markers, etc.). You can retrieve and filter this list as follows. 

In [ ]:
#help(tc.task_event_list)
r = tc.task_event_list(task_id, num_results=10)
for event in r:
    print (event["time"], event["code"], event["is_error"], event["details"])


## Delete files task

File deletion is also an asynchronous task, that is submitted and monitored similar to a transfer task.

In [ ]:
#help(tc.submit_delete)
#create a folder, delete it, wait for completion

# Bookmarks

Bookmarks allow you to keep a list of frequently used endpoints and paths. Full management capabilities (create, retrieve, update, delete) are supported on bookmarks. Note that the REST API itself does not directly support bookmarks when performing operations. It is the responsibility of the client to allow the users to choose bookmarks, and then translate them to endpoint ids to perform ls operations and submit transfers. In particular, the www.globus.org website has full support for bookmarks.

## Create a Bookmark

In [ ]:
bookmark_name = "My Tutorial Bookmark"
endpoint_id = tutorial_endpoint_1
endpoint_path = "/share/godata/"
r = tc.create_bookmark({"endpoint_id":endpoint_id, "path":endpoint_path,"name":bookmark_name})
bookmark_id = r.data['id']
print(r)

## Get a list of bookmarks

In [ ]:
r = tc.bookmark_list()
for b in r:
    print (b.data['name'], b.data['path'], b.data['id'])

## Update a bookmark


In [ ]:
bookmark_data = {
    'name': 'My Updated Tutorial Bookmark'
}
r = tc.update_bookmark(bookmark_id, bookmark_data)
print (r)

## Delete a Bookmark

In [ ]:
r = tc.delete_bookmark(bookmark_id)
print (r)

# Shared endpoints

Shared endpoints are virtual endpoints that refer to a particular "host endpoint" and path, and that allow Globus to manage access control that shared endpoint. Folders on the shared endpoint can be easily shared with other Globus users and groups via access control rules.

## Create a shared endpoint

In [ ]:
# create a dir to share
host_endpoint_id = tutorial_endpoint_1
host_endpoint_path = "/~/shared_dir2"
try:
    r = tc.operation_mkdir(host_endpoint_id, path=host_endpoint_path)
except globus_sdk.GlobusAPIError as e:
    # ignore the error if the directory already exists, otherwise raise
    if "Exists" not in e.code:
        raise

# define the shared endpoint 
shared_ep = {"DATA_TYPE": "shared_endpoint",
             "host_endpoint": host_endpoint_id,
             "host_path": host_endpoint_path,
             "display_name":"My Tutorial Shared Endpoint2",
             # optionally specify additional endpoint fields
             "description": "Test creating a share from globus-jupyter-notebook"
             }

r = tc.create_shared_endpoint(shared_ep)
print("{}: {}".format(r["code"], r["message"]))
print("Endpoin ID", r["id"])
shared_endpoint_id = r["id"]

## Get endpoint information

In [ ]:
r = tc.get_endpoint(shared_endpoint_id)
print("Display name:", r["display_name"])
print("Owner:", r["owner_string"])
print("Host Endpoint ID:", r["host_endpoint_id"])

## Get a list of shared endpoints

In [ ]:
endpoints = tc.endpoint_search(filter_scope="shared-by-me")
print("==== Displaying shared endpoints ===")
for r in endpoints:
    print("{} ({})".format(r["display_name"], r["id"]))


## Add a new access control rule

You can share access to different paths within your shared endpoint with users, groups, or publically. The principal_type can be one of 'identity', 'group, or 'all_authenticated_users', or 'anonymous'.  Each access rule is given a unique access_rule_id, which can be used to manage that access rule.

Here is an example of sharing with the tutorial users group. 

In [ ]:
rule_data = {
    'DATA_TYPE': 'access',
    'permissions': 'rw',
    'principal' : tutorial_users_group,
    'principal_type' : 'group',
    #'principal': 'IDENTITY_ID',
    #'principal_type': 'identity',
    'path': '/'
}

r= tc.add_endpoint_acl_rule(shared_endpoint_id, rule_data)
access_rule_id = r.data['access_id']
print (r)

## Get list of access rules

In [ ]:
r = tc.endpoint_acl_list(shared_endpoint_id)
for a in r:
    print (a.data['id'], a.data['principal_type'], a.data['principal'], a.data['permissions'], a.data['path'])

## Get access rule by id

Get the access rule details using its access_rule_id

In [ ]:
r = tc.get_endpoint_acl_rule(shared_endpoint_id, access_rule_id)
print (r)


## Update access rule

Update an access rule using its access_rule_id.

In [ ]:
rule_data = {
    'DATA_TYPE': 'access',
    'permissions': 'r',
    'principal' : tutorial_users_group,
    'principal_type' : 'group',
    'path': '/'
}
r = tc.update_endpoint_acl_rule(shared_endpoint_id, access_rule_id, rule_data)
print (r)

## Delete access rule

Delete an access rule using its access_rule_id.

In [ ]:
r = tc.delete_endpoint_acl_rule(shared_endpoint_id, access_rule_id)
print (r)